# Load all required packages:

In [1]:
import openslide
import os
import cv2
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import openslide
from PIL import Image
from matplotlib import pyplot as plt
import skimage.measure
import matplotlib.pyplot as plt
import scipy.stats as stats
import torchvision.transforms as transforms
from glob import glob

# Workflow for only ROI (no multiclass labels): Just for reference, probably won't be used (so commented out)
# Parse XML annotation file with X,Y coordinates and instance ID into a DataFrame (only for ROI):


In [2]:
# # First, input xml_filepath and output a dataframe of X,Y coordinates in general. (can be used for ROI as well)
# def xml_to_df(xml_filepath):
#     tree = ET.parse(xml_filepath)
#     root = tree.getroot()
#     append_df = []
#     for index, Annotation in enumerate(root.iter("Annotation")):
#         for Region in Annotation.iter('Region'):
#             x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
#             y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
#             id = np.array([int(Region.get('Id'))])
#             classnames = index + 1
#             coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
#             df = pd.DataFrame(data = coord_dict)
#             df.ID = df.ID.astype(int)
#             append_df.append(df)
#     coord_df = pd.concat(append_df).reset_index(drop=True)
#     return(coord_df)

In [3]:
# Example:
# coord_df = xml_to_df(r'\\fatherserverdw\kyuex\clue images\annotations\roi\2022-06-07 13.18.40.xml')
# coord_df

# Convert X,Y Coordinates to Binary Mask and Read and Resize Image (only for ROI):

In [4]:
# def df_to_image_mask(xml_filepath, ndpi_filepath,downsample_factor):
#     coord_df = xml_to_df(xml_filepath)
#     slide = openslide.open_slide(ndpi_filepath)
#     rgb_dim = slide.dimensions
#     print("Dimension of level 0 of the image is: ",rgb_dim)
#     slide_level_dim = slide.level_dimensions
#     num_levels = len(slide_level_dim)
#     print("Number of levels in this image are:",num_levels)
#     factors = slide.level_downsamples
#     print("Each level is downsampled by:",factors)
#     blank = np.zeros(rgb_dim, dtype = np.uint8) #white
#     for idx, row in coord_df.iterrows():
#         xx = row.X.astype('int')
#         yy = row.Y.astype('int')
#         xy = list(zip(xx,yy))
#         contours = np.array(xy)
#         mask = cv2.fillPoly(blank, pts=[contours.astype(int)], color=idx+125)
#     target_level = slide.get_best_level_for_downsample(downsample_factor)
#     target_dim = slide.level_dimensions[target_level]
#     rsf = [x/y for x,y in zip(rgb_dim,target_dim)]
#     image_resized = slide.read_region(location=(0,0),level=target_level,size=target_dim)
#     mask_resized = cv2.resize(mask,[_//round(rsf[0]) for _ in mask.shape],interpolation=cv2.INTER_NEAREST)
#     #cv2.imwrite(r"\\fatherserverdw\Kevin\\binarymask.jpg",mask_resized) #save binary mask
#     return image_resized,mask_resized

# Cropping RGB Image with Mask (only for ROI):

In [5]:
# Example:
# image,mask = df_to_image_mask(r'\\fatherserverdw\kyuex\clue images\annotations\roi\2022-06-07 13.18.40.xml',r'\\fatherserverdw\kyuex\clue images\2022-06-07 13.18.40.ndpi',10)

In [6]:
# def crop_final_mask(image,mask):
#     imagearr = np.array(image)
#     imagearr = imagearr[:,:,:3]
#     maskarr = np.array(mask)
#     maskrgb = np.repeat(maskarr[:,:,np.newaxis],3,axis=2)
#     for tissueID in range(1,np.max(maskrgb[:])): #from first tissue id to end
#         #masking
#         masktmp = mask==tissueID #boolean
#         masktmp = np.repeat(masktmp[:,:,np.newaxis],3,axis=2) #change shape to match shape of imagearr
#         final_image = np.multiply(imagearr,masktmp)
#         #crop
#         [x_crop,y_crop] = np.where(final_image[:,:,0]>0)
#         cropped_final_image = final_image[np.min(x_crop):np.max(x_crop),np.min(y_crop):np.max(y_crop)]
#         cropped_final_image = cropped_final_image[:,:,::-1]
#         return cropped_final_image

In [7]:
#Example:
# crop_final_mask(image,mask)

# Workflow for Creating Annotation & Its Respective Mask for each Annotation for a WSI Image (for multiple classes):
1. Convert XML file path to X,Y coordinate of contours.
2. Construct a mask with multiple annotation for the entire image, one mask with the same size of the image, with annotations # 1 to 12.
3. Label the binary version of constructed mask from #2 with unique pixel value for each annotation circle.
4. Iterate through the labeled binary mask to crop connected objects
5. Use the cropped coordinates from above #4 to simulataneously crop the constructed mask and the image.
6. Done! You now have saved all of the annotations and its respective masks of the WSI Image!

In [8]:
# First, input xml_filepath and output a dataframe of X,Y coordinates in general. (can be used for ROI as well)
def xml_to_df(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    append_df = []
    for index, Annotation in enumerate(root.iter("Annotation")):
        for Region in Annotation.iter('Region'):
            x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
            y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
            id = np.array([int(Region.get('Id'))])
            classnames = index + 1
            coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
            df = pd.DataFrame(data = coord_dict)
            df.ID = df.ID.astype(int)
            append_df.append(df)
    coord_df = pd.concat(append_df).reset_index(drop=True)
    return(coord_df)

In [9]:
# Then, input xml_path to use xml_to_df function to output X,Y coordinates for each annotation per class:
def coord_to_multiclass_df(xml_path,save_pkl = False):
    coord_df = xml_to_df(xml_path)
    coord_df = coord_df.drop(columns = ["ID"])
    dict = {"corneum" : 1,"spinosum": 2,"hairshaft":3,"hairfollicle":4,"smoothmuscle":5,"oil":6,"sweat":7,"nerve":8,"bloodvessel":9,"ecm":10,"fat":11,"white":12}
    coord_df = coord_df.replace({"ClassNames": dict})
    if save_pkl:
        dst_pth = r'\\fatherserverdw\Kevin\coord_df.pkl'
        coord_df.to_pickle(dst_pth) #save pkl
    return coord_df

In [10]:
# Then input original image and the coord_df to output the mask with unique annotations (1..N, N = 12 in this case):
def create_mask_multi_annot(xml_path, image_path, downsample_factor = 2): #choose downsample factor
    slide = openslide.open_slide(image_path)
    target_level = slide.get_best_level_for_downsample(downsample_factor)
    target_dim = slide.level_dimensions[target_level]
    rsf = [x/y for x,y in zip(slide.dimensions,target_dim)] #resize factor
    mask = np.zeros(target_dim, dtype = np.uint8)
    iter_order = [2,10,5,4,6,11,7,9,8,12,3,1]
    coord_df = coord_to_multiclass_df(xml_path) #use function above

    for i in iter_order:
        coord_df_tmp = coord_df[coord_df.ClassNames == i]
        for idx, row in coord_df_tmp.iterrows():
            xx = row.X.astype('int32')
            yy = row.Y.astype('int32')
            contours = np.array(list(zip(xx,yy)))
            contours = contours/rsf[0]
            class_number = row.ClassNames
            mask = cv2.fillPoly(mask, pts=[contours.astype(int)], color=(int(class_number)))
    return mask

In [11]:
# skimage method:
# Input mask and create binary mask and then output label of connected regions:
# def create_binary_mask_label(xml_path, image_path):
#     mask = create_mask_multi_annot(xml_path, image_path, downsample_factor = 2)
#     binary_mask = mask > 0
#     binary_mask = binary_mask * 255 #255 if labeled (any class from 1 to 12), 0 if unlabeled
#     binary_mask_label = skimage.measure.label(binary_mask,connectivity = 2) #find connected regions, marked from 0 to 128 in the original image
#     return binary_mask_label #returns label of connected regions

In [12]:
# cv2 method:
# Input mask and create binary mask and then output label of connected regions:
def create_binary_mask_label(xml_path, image_path):
    mask = create_mask_multi_annot(xml_path, image_path, downsample_factor = 2)
    binary_mask = mask > 0
    _, binary_mask_label = cv2.connectedComponents(binary_mask.astype(np.uint8))
    return binary_mask_label #returns label of connected regions

In [13]:
# Input label of connected regions and then output the final images and masks:
def create_final_mask_image(xml_path, image_path, dstpath_mask, dstpath_image, downsample_factor = 2, create_new_dstpth = False):
    slide = openslide.open_slide(image_path)
    target_level = slide.get_best_level_for_downsample(downsample_factor)
    target_dim = slide.level_dimensions[target_level]
    image = slide.read_region(location=(0,0),level=target_level,size=target_dim)
    imagearr = np.array(image)
    imagearr = imagearr[:,:,:3]
    binary_mask_label = create_binary_mask_label(xml_path, image_path)
    mask = create_mask_multi_annot(xml_path, image_path, downsample_factor = 2)
    for idx,label in enumerate(range(1,np.max(binary_mask_label)+1)):
        boo = binary_mask_label == label
        boolabel = boo * label
        loca = np.where(boolabel == label)
        x = loca[0]
        y = loca[1]
        targetmask = mask[min(x):max(x),min(y):max(y)]
        if create_new_dstpth:
            dstpth = dstpath_mask + str(idx)+'.png'
        dstpth = dstpath_mask
        Image.fromarray(targetmask).save(dstpth)
        targetim = imagearr[min(x):max(x),min(y):max(y),:]
        if create_new_dstpth:
            dstpth1 = dstpath_image + str(idx)+'.png'
        dstpth1 = dstpath_image
        Image.fromarray(targetim).save(dstpth1)

# To get the respective annotation images and masks for all xml's in the file path: \\fatherserverdw\kyuex\image\CLUE\3D study\he\c1

In [14]:
# first get relevant paths:
query_src_path = r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\c1'

xml_complete_path = glob(os.path.join(query_src_path,'*.xml'))
xml_files_path =  [_ for _ in os.listdir(query_src_path) if _.endswith(".xml")]
xml_image_name = [x.replace('.xml','') for x in xml_files_path]

# only subset wsi images that have xml files and change file path accordingly (meaning filter out images that don't have annotations):
overlap_ndpi = [_ for _  in os.listdir(query_src_path) if (_.endswith('ndpi') and os.path.splitext(_)[0] in xml_image_name)]
wsi_complete_path = [os.path.join(query_src_path,x) for x in overlap_ndpi]
wsi_files_path = [os.path.basename(_) for _ in wsi_complete_path]
png_save_name = [x.replace('.ndpi','.png') for x in wsi_files_path]

In [ ]:
# first define dstpth:
dstpth_image = r'\\fatherserverdw\Kevin\kevin_train\dataset 221122\image'
dstpth_mask = r'\\fatherserverdw\Kevin\kevin_train\dataset 221122\mask'
for idx in range(len(wsi_complete_path)):
    new_png_save_name = png_save_name[idx].replace('C1','C1' + '_' + str(idx))
    tmp_dstpath_mask = os.path.join(dstpth_mask,new_png_save_name)
    tmp_dstpath_image = os.path.join(dstpth_image,new_png_save_name)
    create_final_mask_image(xml_path = xml_complete_path[idx], image_path = wsi_complete_path[idx], dstpath_mask = tmp_dstpath_mask, dstpath_image = tmp_dstpath_image, downsample_factor=2, create_new_dstpth = False)

# Then, workflow to randomly (from a gaussian distribution) place all images & masks from the output images & masks (from above workflow) on top of a 5000 x 5000 image tile to create the training/validation set for 2D skin DL model:
It should look similar to the files in the path (example): fatherserverdw\Q\research\images\skin_aging\deeplab_trainingset\v11\training\big_tiles


In [ ]:
def rotate_image_cv2(mat, angle):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    Image has to be uint8
    """

    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0])
    abs_sin = abs(rotation_mat[0,1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h),	flags = cv2.INTER_NEAREST)
    return rotated_mat

In [ ]:
image_tile = np.zeros((5120,5120)) #black image tile
image_tile = np.repeat(image_tile[:,:,np.newaxis],3,axis=2)
image_tile = Image.fromarray((image_tile*255).astype(np.uint8))

msk_tile = np.zeros((5120,5120)) #black msk tile
# msk_tile = np.repeat(msk_tile[:,:,np.newaxis],3,axis=2)
msk_tile = Image.fromarray((msk_tile*255).astype(np.uint8))

# pick one file to work with first:
number_of_files = len(os.listdir(r'\\fatherserverdw\Kevin\targetimage\immasked'))
rand_file_num = np.random.randint(1,number_of_files-1,size = number_of_files)
lower = 0
upper = 5120
mu = 2560
sigma = 1280
rand_coord = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc = mu, scale = sigma)
# plt.hist(rand_coord_pair.rvs(10000))
# plt.show()
rand_coord_num = number_of_files+1
count = 0

while count < 5000:
    for idx in np.nditer(rand_file_num):
        image = Image.open(r'\\fatherserverdw\Kevin\targetimage\immasked\\' + str(rand_file_num[idx]) + '.png')
        msk = Image.open(r'\\fatherserverdw\Kevin\targetmask\\' + str(rand_file_num[idx]) + '.png')
        rand_coord_x = np.round(rand_coord.rvs(rand_coord_num))
        rand_coord_x = [int(x) for x in rand_coord_x]
        rand_coord_y = np.round(rand_coord.rvs(rand_coord_num))
        rand_coord_y = [int(y) for y in rand_coord_y]
        x = rand_coord_x[idx]
        y = rand_coord_y[idx]

        rotationangle = np.random.randint(0,360)
        image1 = np.array(image)
        imagerot = rotate_image_cv2(image1,rotationangle)
        msk2 = np.array(msk)
        maskrot = rotate_image_cv2(msk2,rotationangle)

        contrast_range = np.random.uniform(low=0,high=0.1)
        brightness_range = np.random.uniform(low = 0, high = 0.1)
        hue_range = np.random.uniform(low = 0, high = 0.01)
        saturation_range = np.random.uniform(low=0,high=0.1)

        color_transform = transforms.ColorJitter(brightness=brightness_range, contrast=contrast_range, saturation=saturation_range, hue=hue_range)
        imagerot2 = color_transform(Image.fromarray(imagerot))
        maskrot2 = maskrot
        maskrot = Image.fromarray(maskrot)
        msk_tile.paste(maskrot,(x,y))#, mask=maskrot)
        maskrot2[maskrot2>0] = 255
        maskrot2 = Image.fromarray(maskrot2)
        image_tile.paste(imagerot2,(x,y), mask=maskrot2)
        count += 1

image_tile.show()
msk_tile.show()

In [ ]:
# split into 1000 x 1000 tiles and generate training data image/mask:

image_tile = np.array(image_tile)
M = image_tile.shape[0]//5
N = image_tile.shape[1]//5
image_tiles_split = [image_tile[x:x+M,y:y+N] for x in range(0,image_tile.shape[0],M) for y in range(0,image_tile.shape[1],N)]

msk_tile = np.array(msk_tile)
M = msk_tile.shape[0]//5
N = msk_tile.shape[1]//5
msk_tiles_split = [msk_tile[x:x+M,y:y+N] for x in range(0,msk_tile.shape[0],M) for y in range(0,msk_tile.shape[1],N)]

for idx in range(len(msk_tiles_split)):
    imgsave = image_tiles_split[idx]
    cv2.imwrite(r"\\fatherserverdw\Kevin\kevin_train\training\im\\" + str(idx+2) + ".png",imgsave)
    msksave = msk_tiles_split[idx]
    cv2.imwrite(r"\\fatherserverdw\Kevin\kevin_train\training\label\\" + str(idx+201) + ".png",msksave)


In [ ]:
# # # split into 1000 x 1000 tiles and generate validation data for image/mask, REMEMBER TO RUN THE RANDOMLY GENERATED IMAGE FIRST:
# image_tile = np.array(image_tile)
# M = image_tile.shape[0]//5
# N = image_tile.shape[1]//5
# image_tiles_split = [image_tile[x:x+M,y:y+N] for x in range(0,image_tile.shape[0],M) for y in range(0,image_tile.shape[1],N)]
#
# msk_tile = np.array(msk_tile)
# M = msk_tile.shape[0]//5
# N = msk_tile.shape[1]//5
# msk_tiles_split = [msk_tile[x:x+M,y:y+N] for x in range(0,msk_tile.shape[0],M) for y in range(0,msk_tile.shape[1],N)]
#
# for idx in range(len(msk_tiles_split)):
#     imgsave = image_tiles_split[idx]
#     cv2.imwrite(r"\\fatherserverdw\Kevin\kevin_train\validation\im\\" + str(idx+76) + ".png",imgsave)
#     msksave = msk_tiles_split[idx]
#     cv2.imwrite(r"\\fatherserverdw\Kevin\kevin_train\validation\label\\" + str(idx+76) + ".png",msksave)